In [1]:
import networkx as nx
from operator import itemgetter
from networkx import ego_graph
import matplotlib.pyplot as plt
import pylab 
import random
import pandas as pd
from networkx.algorithms.assortativity.mixing import mixing_dict
from networkx.utils import dict_to_numpy_array
from networkx.utils import accumulate
import scipy.stats as stats
import numpy as np  
#import pylab as pl
from pylab import *  
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

import random


## Create a graph

In [2]:
def createGraph(filename) :
    G = nx.Graph()
    for line in open(filename) :
        strlist = line.split()
        n1 = int(strlist[0])
        n2 = int(strlist[1])
        G.add_edges_from([(n1, n2)]) #G.add_edges_from([(n1, n2)])
    return G

In [3]:
import csv
#G= createGraph('11ER.csv')
G=nx.read_edgelist("assortative_protein_3.edgelist", nodetype=int)



In [4]:
# a set of nodes which degree is equal to seleted node 

def equal_degree_nodes(G,node):
    degree_dist=sorted(G.degree_iter(),key=itemgetter(1),reverse=True)
    ad=[]
    for x,y in degree_dist:
        if y == G.degree(node):
            ad.append(x)
    return ad

In [5]:
#A function for degree sequences

def node_neighbor_max_degree_xy(G, x='out', y='in', weight=None, nodes=None):
    if nodes is None:
        nodes = set(G)
    else:
        nodes = set(nodes)
    xdeg = G.degree
    ydeg = G.degree
    for start_node_ID in xdeg(nodes):
        neighbors = (nbr for _,nbr in G.edges(start_node_ID) if nbr in nodes)
        
        for end_node_ID in neighbors:
            nbrdeg_start = G.degree(G[start_node_ID])
            nbrdeg_end = G.degree(G[end_node_ID])     
            nbrdeg_start.pop(end_node_ID)
            nbrdeg_end.pop(start_node_ID)
                       
            a = []
            b = []
            
            for n in nbrdeg_start:                
                a.append(nbrdeg_start[n])
            if a:
                degu = max(a)-1 
            else:
                degu = 1
            for m in nbrdeg_end:
                b.append(nbrdeg_end[m])
            if b:
                degv = max(b)-1
            else:
                degv = 1
            
            yield( start_node_ID,end_node_ID,degu,degv)

In [6]:
#A function for checking new neighbour maximum degree

def new_maxi_degree(G,start_node_ID,end_node_ID):
    nodes = set(G)

    nbrdeg_start = G.degree(G[start_node_ID])
    nbrdeg_end = G.degree(G[end_node_ID])
            
    nbrdeg_start.pop(end_node_ID)
    nbrdeg_end.pop(start_node_ID)
           
    a = []
    b = []
    for n in nbrdeg_start:                
        a.append(nbrdeg_start[n])
    if a:
        degu = max(a)-1 
    else:
        degu = 1
    for m in nbrdeg_end:
        b.append(nbrdeg_end[m])
    if b:
        degv = max(b)-1
    else:
        degv = 1
            
    return( start_node_ID,end_node_ID,degu,degv)        


In [7]:
#initial original second-order assortative coefficient

node_neg_maxi_degree=node_neighbor_max_degree_xy(G)
a1,b1,x1,y1=zip(*node_neg_maxi_degree)
second_order_coef = stats.pearsonr(x1,y1)[0]
print(second_order_coef)
a1=list(a1)
b1=list(b1)
x1=list(x1)
y1=list(y1)
second_order_coef = stats.pearsonr(x1,y1)[0]
print(second_order_coef)

0.866876244447
0.866876244447


In [8]:

count_number=[0]  #record the times of rewiring
sec_order_coefficient = []

import time

start = time.time()

for n in range(300000):
    
    #print(i)
    if n == 0:
        node_neg_maxi_degree=node_neighbor_max_degree_xy(G)
        node_s2,node_e2,maxi_x2,maxi_y2=zip(*node_neg_maxi_degree)
        second_order_coef = stats.pearsonr(maxi_x2,maxi_y2)[0]
        print(second_order_coef)
        sec_order_coefficient.append(second_order_coef)
    
    a,x = random.choice(G.edges())  #randomly choose an edge from network

    A = random.choice([a,x]) #randomly choose a node from this edge for next step KA = KB

    X = x if A == a else a  #for constant A and X
    #print(A,X)
    
    B ,Y = random.choice(G.edges(equal_degree_nodes(G,A))) # another edge where KA = KB
    
    #print(B,Y)
    
    if A == B:
        pass
    
    elif X == Y:
        pass
    
    elif A == Y:
        pass
    
    elif B == X:
        pass
    
    else:
        
        if (G.has_edge(A,Y) == False and G.has_edge(B,X) == False):
            
            count_number[0]=count_number[0]+1
            #print(len(count_number))
            #print(A,X)
            #print(B,Y)
            #print(G.has_edge(A,Y),G.has_edge(B,X),G.has_edge(A,X),G.has_edge(B,Y))
            a1=list(a1)
            b1=list(b1)
            x1=list(x1)
            y1=list(y1)
            a2=tuple(a1)
            b2=tuple(b1)
            x2=tuple(x1)
            y2=tuple(y1)
            
            
            second_order_coef_b = stats.pearsonr(x1,y1)[0]    #calculating original second-order coefficient
            
            G.remove_edge(A,X)      #rewiring
            G.remove_edge(B,Y)
            G.add_edge(A,Y)
            G.add_edge(B,X)
            
            for i in range(len(x1)):          #change neighbour maximum degree and nodes
                if a1[i] == A and b1[i]==X:
                    b1[i]=Y
                    a1[i],b1[i],x1[i],y1[i] = new_maxi_degree(G,a1[i],b1[i])
        
                if a1[i] == B and b1[i]==Y:
                    b1[i]=X
                    a1[i],b1[i],x1[i],y1[i] = new_maxi_degree(G,a1[i],b1[i])
    
                if a1[i] == X and b1[i]==A:
                    b1[i]=B
                    a1[i],b1[i],x1[i],y1[i] = new_maxi_degree(G,a1[i],b1[i])
        
                if a1[i] == Y and b1[i]==B:
                    b1[i]=A
                    a1[i],b1[i],x1[i],y1[i] = new_maxi_degree(G,a1[i],b1[i])
                
                if a1[i] == A or a1[i] == X or a1[i] == B or a1[i] == Y:
                    #print(a1[i],b1[i])
                    a1[i],b1[i],x1[i],y1[i] = new_maxi_degree(G,a1[i],b1[i])
                if b1[i] == A or b1[i] == X or b1[i] == B or b1[i] == Y:
                    #print(a1[i],b1[i])
                    a1[i],b1[i],x1[i],y1[i] = new_maxi_degree(G,a1[i],b1[i])

        
            second_order_coef_af = stats.pearsonr(x1,y1)[0]   #calculating the second-order assortative coefficient after rewiring
            
            if second_order_coef_b <= second_order_coef_af:   #disassortative case selects small one
                G.remove_edge(A,Y)
                G.remove_edge(B,X)
                G.add_edge(A,X)
                G.add_edge(B,Y)
                
                a1,b1,x1,y1=a2,b2,x2,y2
                
            else:
                pass
               
    if count_number[0]!= 0 and count_number[0] in range(0,300001,1000):  #record second-order assortative coefficient every 1000 times
        
        if second_order_coef_b <= second_order_coef_af:
            sec_order_coefficient.append(second_order_coef_b)
            print(second_order_coef_b)
        else:
            sec_order_coefficient.append(second_order_coef_af)
            print(second_order_coef_af)
            
    
    if len(sec_order_coefficient)>2 and (abs(sec_order_coefficient[-1] - sec_order_coefficient[-2]) < 0.000001) and (sec_order_coefficient[-1] != sec_order_coefficient[-2]):
        
        nx.write_edgelist(G, "disassortative_ER_final.edgelist")
        break
        
    if count_number[0]==200000:
        break
        
    

nx.write_edgelist(G, "assortative_protein_3_2.edgelist")
        
with open('assortative_protein_3_2.csv','w') as f:
    f_csv = csv.writer(f)
    for item in sec_order_coefficient:
        f_csv.writerow([item])

end = time.time()
print("cost:", end-start)
        
        





0.866876244447
cost: 2.1724908351898193
